In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install optuna

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import optuna
from tqdm import tqdm
import lightgbm as lgb
from sklearn.metrics import mean_tweedie_deviance,r2_score,mean_squared_error

In [ ]:
full_grid=pd.read_pickle('/content/drive/MyDrive/Case_study1/store_wise/CA_1_full_grid.pkl')
full_grid['d'] = full_grid['d'].apply(lambda x: x[2:]).astype(np.int16)
full_grid['item_id']=LabelEncoder().fit_transform(full_grid['item_id']).astype('int16')
full_grid['dept_id']=LabelEncoder().fit_transform(full_grid['dept_id']).astype('int16')
full_grid['cat_id']=LabelEncoder().fit_transform(full_grid['cat_id']).astype('int16')

In [ ]:
full_grid.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'release', 'wm_yr_wk_x', 'sell_price', 'price_max',
       'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique',
       'item_nunique', 'price_momentum', 'price_momentum_m',
       'price_momentum_y', 'wm_yr_wk_y', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y',
       'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std',
       'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean',
       'enc_item_id_std', 'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32',
       'lag_33', 'lag_34', 'lag_35', 'lag_36', 'lag_37', 'lag_38', 'lag_39',
       'lag_40', 'lag_41', 'lag_42', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30',
       'rolling_std_30', 'rolling_mean_60', 'rolling_std_60',
       'rolling_mean

In [ ]:
features=['item_id', 'dept_id', 'cat_id','d',
        'release', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
       'price_momentum', 'price_momentum_m', 'price_momentum_y', 'wday',
       'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m',
       'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean',
       'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std',
       'enc_item_id_mean', 'enc_item_id_std', 'lag_28', 'lag_29', 'lag_30',
       'lag_31', 'lag_32', 'lag_33', 'lag_34', 'lag_35', 'lag_36', 'lag_37',
       'lag_38', 'lag_39', 'lag_40', 'lag_41', 'lag_42', 'rolling_mean_7',
       'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30',
       'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'AWND', 'PRCP',
       'TAVG']

target='sales'

tr_mask=(full_grid['d']>=710) & (full_grid['d']<=1550)

vl_mask=(full_grid['d']>=1551) & (full_grid['d']<=1579)


train_data = lgb.Dataset(full_grid[tr_mask][features], 
                       label=full_grid[tr_mask]['sales'])

valid_data = lgb.Dataset(full_grid[vl_mask][features], 
                           label=full_grid[vl_mask]['sales'])

In [ ]:
def objective(trial):

  param = {
        'objective': 'tweedie',
        'metric': 'rmse',
        'tweedie_variance_power': 1.1,
        'verbose': -1,
        'boosting_type': 'gbdt',
        'boost_from_average':False,
        'subsample' : trial.suggest_uniform('subsample', 0.5, 1.0),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.001,0.1),
        'num_leaves': trial.suggest_int('num_leaves', 500,4000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'n_estimators': trial.suggest_int('n_estimators',100,3000),
        'min_data_in_leaf': trial.suggest_int('num_leaves', 500,4000),
        'feature_pre_filter':False
    }
    
  gbm = lgb.train(param,train_data)
  y_pred=gbm.predict(full_grid[vl_mask][features])
  
  print('r2_score: ',r2_score(valid_data.get_label(),y_pred))
  print('rmse: ',mean_squared_error(valid_data.get_label(),y_pred,squared=False))

  return mean_tweedie_deviance(valid_data.get_label(),y_pred,power=1.1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
print(study.best_trial)